# SciPy.org's [NumPy](http://www.numpy.org/)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Looking-for-information" data-toc-modified-id="Looking-for-information-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Looking for information</a></span></li><li><span><a href="#Data-types" data-toc-modified-id="Data-types-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://docs.scipy.org/doc/numpy/user/basics.types.html" target="_blank">Data types</a></a></span></li><li><span><a href="#Array-creation,-allocation-and-initialization" data-toc-modified-id="Array-creation,-allocation-and-initialization-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://docs.scipy.org/doc/numpy/user/quickstart.html#array-creation" target="_blank">Array creation, allocation and initialization</a></a></span><ul class="toc-item"><li><span><a href="#Allocation-(without-initializing-the-data-array)" data-toc-modified-id="Allocation-(without-initializing-the-data-array)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Allocation (without initializing the data array)</a></span></li><li><span><a href="#Allocating-and-initalizing" data-toc-modified-id="Allocating-and-initalizing-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Allocating and initalizing</a></span><ul class="toc-item"><li><span><a href="#Using-a-list" data-toc-modified-id="Using-a-list-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Using a list</a></span></li><li><span><a href="#Using-&quot;initializers&quot;" data-toc-modified-id="Using-&quot;initializers&quot;-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Using "initializers"</a></span></li></ul></li><li><span><a href="#Defining-types" data-toc-modified-id="Defining-types-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Defining <a href="https://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html" target="_blank">types</a></a></span></li><li><span><a href="#Reshaping" data-toc-modified-id="Reshaping-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span><a href="https://docs.scipy.org/doc/numpy/user/quickstart.html#shape-manipulation" target="_blank">Reshaping</a></a></span></li><li><span><a href="#C-vs-Fortran-order" data-toc-modified-id="C-vs-Fortran-order-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>C vs Fortran order</a></span></li><li><span><a href="#Creation-(without-allocation-memory)" data-toc-modified-id="Creation-(without-allocation-memory)-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Creation (without allocation memory)</a></span></li></ul></li><li><span><a href="#Views-and-copies" data-toc-modified-id="Views-and-copies-5"><span class="toc-item-num">5&nbsp;&nbsp;</span><a href="https://docs.scipy.org/doc/numpy/user/quickstart.html#copies-and-views" target="_blank">Views and copies</a></a></span><ul class="toc-item"><li><span><a href="#Pointer-copy" data-toc-modified-id="Pointer-copy-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Pointer copy</a></span></li><li><span><a href="#Shallow-copy-(view)" data-toc-modified-id="Shallow-copy-(view)-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Shallow copy (view)</a></span></li><li><span><a href="#Deep-(object-+--data)-copy" data-toc-modified-id="Deep-(object-+--data)-copy-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Deep (object +  data) copy</a></span></li></ul></li><li><span><a href="#More-about-array-indexing-and-iterating" data-toc-modified-id="More-about-array-indexing-and-iterating-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>More about array <a href="https://docs.scipy.org/doc/numpy/user/basics.indexing.html" target="_blank">indexing</a> and <a href="https://docs.scipy.org/doc/numpy/user/quickstart.html#indexing-slicing-and-iterating" target="_blank">iterating</a></a></span><ul class="toc-item"><li><span><a href="#More-about-simple-indexing" data-toc-modified-id="More-about-simple-indexing-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>More about simple indexing</a></span></li><li><span><a href="#Array-indexing-(also-known-as-fancy-indexing)" data-toc-modified-id="Array-indexing-(also-known-as-fancy-indexing)-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span><a href="https://docs.scipy.org/doc/numpy/user/quickstart.html#indexing-with-arrays-of-indices" target="_blank">Array indexing</a> (also known as <a href="https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#advanced-indexing" target="_blank">fancy indexing</a>)</a></span></li><li><span><a href="#Boolean-array-indexing" data-toc-modified-id="Boolean-array-indexing-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Boolean array indexing</a></span></li><li><span><a href="#Iterating" data-toc-modified-id="Iterating-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Iterating</a></span></li></ul></li><li><span><a href="#Extending-arrays" data-toc-modified-id="Extending-arrays-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Extending arrays</a></span></li><li><span><a href="#Permuting-(swapping)-dimensions-(axes)" data-toc-modified-id="Permuting-(swapping)-dimensions-(axes)-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Permuting (swapping) dimensions (axes)</a></span></li><li><span><a href="#Increasing-and-decreasing-dimensions" data-toc-modified-id="Increasing-and-decreasing-dimensions-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Increasing and decreasing dimensions</a></span></li><li><span><a href="#Slicing" data-toc-modified-id="Slicing-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Slicing</a></span></li><li><span><a href="#Mathematics" data-toc-modified-id="Mathematics-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Mathematics</a></span><ul class="toc-item"><li><span><a href="#Some-school-math" data-toc-modified-id="Some-school-math-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Some <em>school</em> math</a></span></li><li><span><a href="#Some-measurements" data-toc-modified-id="Some-measurements-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Some measurements</a></span></li><li><span><a href="#Some-vector-math" data-toc-modified-id="Some-vector-math-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>Some vector math</a></span></li><li><span><a href="#Some-matrix-math" data-toc-modified-id="Some-matrix-math-11.4"><span class="toc-item-num">11.4&nbsp;&nbsp;</span>Some matrix math</a></span></li></ul></li><li><span><a href="#Broadcasting" data-toc-modified-id="Broadcasting-12"><span class="toc-item-num">12&nbsp;&nbsp;</span><a href="https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html" target="_blank">Broadcasting</a></a></span></li><li><span><a href="#How-fast-is-Numpy's-array-math?" data-toc-modified-id="How-fast-is-Numpy's-array-math?-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>How fast is Numpy's array math?</a></span></li><li><span><a href="#Arrays-of-objects" data-toc-modified-id="Arrays-of-objects-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Arrays of objects</a></span></li><li><span><a href="#Structured-arrays" data-toc-modified-id="Structured-arrays-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Structured arrays</a></span></li><li><span><a href="#Disk-I/O" data-toc-modified-id="Disk-I/O-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>Disk I/O</a></span><ul class="toc-item"><li><span><a href="#Endianness" data-toc-modified-id="Endianness-16.1"><span class="toc-item-num">16.1&nbsp;&nbsp;</span><a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.dtype.byteorder.html" target="_blank">Endianness</a></a></span></li></ul></li><li><span><a href="#Timming-NumPy?" data-toc-modified-id="Timming-NumPy?-17"><span class="toc-item-num">17&nbsp;&nbsp;</span>Timming NumPy?</a></span></li></ul></div>

## Intro

NumPy extends lists in Python when we need to work with arrays of numbers, [providing](https://www.oreilly.com/library/view/python-for-data/9781449323592/ch04.html) a higher performance (throught [vectorization](https://en.wikipedia.org/wiki/Array_programming)) and [functionality](https://numpy.org/doc/stable/reference/index.html) (basically, [Linear Algebra](https://en.wikipedia.org/wiki/Linear_algebra#:~:text=Linear%20algebra%20is%20the%20branch,almost%20all%20areas%20of%20mathematics.)).

In [1]:
!pip install numpy
import numpy as np


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


## Looking for information

In [2]:
np.lookfor("invert")

Search results for 'invert'
---------------------------
numpy.bitwise_not
    Compute bit-wise inversion, or bit-wise NOT, element-wise.
numpy.matrix.getI
    Returns the (multiplicative) inverse of invertible `self`.
numpy.array_api.bitwise_invert
    Array API compatible wrapper for :py:func:`np.invert <numpy.invert>`.
numpy.in1d
    Test whether each element of a 1-D array is also present in a second array.
numpy.isin
    Calculates ``element in test_elements``, broadcasting over `element` only.
numpy.quantile
    Compute the q-th quantile of the data along the specified axis.
numpy.transpose
    Returns an array with axes transposed.
numpy.percentile
    Compute the q-th percentile of the data along the specified axis.
numpy.array_api._array_object.Array.__invert__
    Performs the operation __invert__.
numpy.nanquantile
    Compute the qth quantile of the data along the specified axis,
numpy.nanpercentile
    Compute the qth percentile of the data along the specified axis,
numpy.l

* Using IPython, remember that it's possible to use the tabulator to extend some command or to use a wildcard to get information about the numpy's stuff:

In [3]:
np.*?

## [Data types](https://docs.scipy.org/doc/numpy/user/basics.types.html)
NumPy can works (create, operate and I/O) with arrays containig:
1. Signed and unsigned integers of 8, 16, 32 and 64 bits.
2. Floating point numbers of 32 and 64 bits.
3. Complex (floating point) numbers of 64 and 128 bits.
4. Strings.

## [Array creation, allocation and initialization](https://docs.scipy.org/doc/numpy/user/quickstart.html#array-creation)

### A word about efficiency
NumPy has been designed to be efficient when it use blocks of memory (possiblely contiguous) of constant size. In other words, when the structures are static.

### Allocation (without initializing the data array)

In [4]:
A = np.empty(12)
print(A) # Uninitialized garbage values, possiblely at random.

[ 0.          0.6893465  -0.         -0.96220197 -0.         -0.4933043
  0.          0.33553948  0.          0.00658427  0.          0.35510008]


In [5]:
print(np.empty_like(A))

[ 0.          0.6893465  -0.         -0.96220197 -0.         -0.4933043
  0.          0.33553948  0.          0.00658427  0.          0.35510008]


### Allocating and initalizing

#### Using a list

In [6]:
l = [1, 2, 3]
type(l)

list

In [7]:
A = np.array(l)
A

array([1, 2, 3])

In [8]:
A = np.array([[1,1.0],(1+1j,.3)])
print(A)

[[1. +0.j 1. +0.j]
 [1. +1.j 0.3+0.j]]


#### Using "initializers"

In [9]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
np.zeros((5,5))

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [11]:
np.ones(10)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [12]:
np.full((5,5), 2)

array([[2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2]])

In [13]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [14]:
np.linspace(1., 4., 6)

array([1. , 1.6, 2.2, 2.8, 3.4, 4. ])

In [15]:
np.random.rand(10)

array([0.69709817, 0.13721813, 0.68189834, 0.35580648, 0.35376678,
       0.49541702, 0.95493675, 0.71974844, 0.35800054, 0.0129539 ])

In [16]:
# The random number generator is not reset in each call
print(np.random.random((5,5)))
print(np.random.random((5,5)))

[[0.5770763  0.01264241 0.63677057 0.97090763 0.61459828]
 [0.47949033 0.62235514 0.75981494 0.72037019 0.81613719]
 [0.2989445  0.45221349 0.87254846 0.97332779 0.56301019]
 [0.61550441 0.44924716 0.12420157 0.67909247 0.67641966]
 [0.10217629 0.44080091 0.40679546 0.85487261 0.31471108]]
[[0.94511915 0.81830483 0.54756443 0.76205471 0.07145301]
 [0.70481423 0.08875165 0.76212969 0.22092896 0.17573352]
 [0.22913461 0.34392863 0.6519354  0.32614281 0.25533012]
 [0.53262988 0.00164934 0.41988126 0.57052458 0.1703233 ]
 [0.5200494  0.33740782 0.51690989 0.11462744 0.36736289]]


In [17]:
print(np.eye(5)) # Identity matrix

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [18]:
A = np.array([i for i in range(5)]) # 1-D list comprehension
print(A, A[1], A.shape)

[0 1 2 3 4] 1 (5,)


In [19]:
A = np.array([[j+i*4 for j in range(4)] for i in range(5)]) # 2-D list comprehension
print(A, A.shape)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]] (5, 4)


In [20]:
# This is efficient

import time
a = np.linspace(0, 10_000_000, num=10_000_000 + 1)
t1=time.time()
a = np.sin(a)
t2 = time.time()
print("With linspace and ufunc it took {} seconds for WAY MORE values!".format(t2-t1))
a[:5]

With linspace and ufunc it took 0.06341910362243652 seconds for WAY MORE values!


array([ 0.        ,  0.84147098,  0.90929743,  0.14112001, -0.7568025 ])

### Defining [types](https://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html)

In [21]:
A = np.array([1, 2], dtype=np.uint8)
A

array([1, 2], dtype=uint8)

In [22]:
type(A[0])

numpy.uint8

### [Reshaping](https://docs.scipy.org/doc/numpy/user/quickstart.html#shape-manipulation)

In [23]:
A = np.array([[1,2,3],[4,5,6]])
print(A)
print(A.shape)

[[1 2 3]
 [4 5 6]]
(2, 3)


In [24]:
np.reshape(A, (3, 2))

array([[1, 2],
       [3, 4],
       [5, 6]])

### C vs Fortran order

In [25]:
A = np.empty((5,4), dtype=np.int16, order='C') # C order is the default
for row in range(A.shape[0]):
    for col in range(A.shape[1]):
        A[row, col] = col+row*A.shape[1]
print(A)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]]


In [26]:
print(np.isfortran(A))

False


In [27]:
B = np.reshape(A, (4,5), order='F')
print(B)

[[ 0 16 13 10  7]
 [ 4  1 17 14 11]
 [ 8  5  2 18 15]
 [12  9  6  3 19]]


In [28]:
print(np.isfortran(B))

True


In [29]:
print(A.flags)
print(B.flags)

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



### Dynamic creation
Be careful, dynamic arrays in NumPy are slower than static ones.

* Creating an empty (unallocated) array for working with 8-bits unsigned integers:

In [30]:
A = np.array([], dtype=np.uint8)
print(A, A.dtype)
A

[] uint8


array([], dtype=uint8)

In [31]:
A = np.concatenate((A, np.array([1, 2], dtype=np.uint8)))
A

array([1, 2], dtype=uint8)

(see NumPy's data [types](https://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html))

* NumPy arrays are objects of the "numpy.ndarray" class:

In [32]:
print(type(A))

<class 'numpy.ndarray'>


* All arrays has at least one dimension and a length in each dimenssion.

In [33]:
print(f"number of dimensions={A.ndim}, shape={A.shape}")

number of dimensions=1, shape=(2,)


## [Views and copies](https://docs.scipy.org/doc/numpy/user/quickstart.html#copies-and-views)

### Pointer copy

* Simple assignments do not copy of array objects or of their data:

In [34]:
# https://stackoverflow.com/questions/56090021/list-comprehension-python-prime-numbers
Primes_less_than_100 = np.array([x for x in range(2,100) if not any([x % y == 0 for y in range(2, int(x/2)+1)])])
Primes_less_than_100

array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,
       61, 67, 71, 73, 79, 83, 89, 97])

In [35]:
A = Primes_less_than_100 # This is a copy of pointers
A

array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,
       61, 67, 71, 73, 79, 83, 89, 97])

In [36]:
A[0]=1
A

array([ 1,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,
       61, 67, 71, 73, 79, 83, 89, 97])

In [37]:
Primes_less_than_100

array([ 1,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,
       61, 67, 71, 73, 79, 83, 89, 97])

In [38]:
id(A)

139691707028208

In [39]:
id(Primes_less_than_100)

139691707028208

In [40]:
A is Primes_less_than_100

True

### Shallow copy (view)

In [41]:
print(Primes_less_than_100.shape)

(25,)


In [42]:
A = Primes_less_than_100.reshape(5,5)
print(A.shape)

(5, 5)


In [43]:
A is Primes_less_than_100

False

In [44]:
A.base is Primes_less_than_100

True

In [45]:
A.flags.owndata

False

In [46]:
print(Primes_less_than_100, A)

[ 1  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97] [[ 1  3  5  7 11]
 [13 17 19 23 29]
 [31 37 41 43 47]
 [53 59 61 67 71]
 [73 79 83 89 97]]


In [47]:
A[0,0] = 2
print(Primes_less_than_100, A)

[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97] [[ 2  3  5  7 11]
 [13 17 19 23 29]
 [31 37 41 43 47]
 [53 59 61 67 71]
 [73 79 83 89 97]]


### Deep (object +  data) copy

In [48]:
Primes_less_than_100 = np.array([x for x in range(2,100) if not any([x % y == 0 for y in range(2, int(x/2)+1)])])
A = np.copy(Primes_less_than_100)

In [49]:
A is Primes_less_than_100

False

In [50]:
Primes_less_than_100[0]

2

In [51]:
A[0] = 1

In [52]:
print(Primes_less_than_100[0], A[0])

2 1


In [53]:
timeit A=Primes_less_than_100 # This is much faster, depending on the size of the array

10 ns ± 0.0857 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [54]:
%timeit A = np.copy(Primes_less_than_100)

442 ns ± 3.53 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## More about array [indexing](https://docs.scipy.org/doc/numpy/user/basics.indexing.html) and [iterating](https://docs.scipy.org/doc/numpy/user/quickstart.html#indexing-slicing-and-iterating)
NumPy arrays are indexed by nonnegative integers.

### More about simple indexing

In [55]:
A = np.arange(20).reshape(5, 4)
print(A)
print(A[1, 2]) # [row, column]

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]]
6


In [56]:
print(A[1]) # Get the second row

[4 5 6 7]


In [57]:
A[1][2] # [row][column]. Access first to the row, and then, to the column

6

* Be careful:

In [58]:
timeit A[1][2]

96 ns ± 0.58 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [59]:
timeit A[1,2] # Access directly to the element

48 ns ± 0.447 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### [Array indexing](https://docs.scipy.org/doc/numpy/user/quickstart.html#indexing-with-arrays-of-indices) (also known as [fancy indexing](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#advanced-indexing))

In [60]:
A = np.arange(20).reshape(5, 4)
print(A)
dim_0_coordinates = [0, 1, 2]
dim_1_coordinates = [3, 2, 1]
print(A[dim_0_coordinates, dim_1_coordinates])

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]]
[3 6 9]


* Notice that the indices can be determined by lists:

In [61]:
print(A[[i for i in range(3)], [i for i in range(3,0,-1)]])

[3 6 9]


* ... or by NumPy arrays:

In [62]:
print(np.arange(3))
print(np.arange(3,0,-1))
print(A[np.arange(3), np.arange(3,0,-1)])

[0 1 2]
[3 2 1]
[3 6 9]


In [63]:
A = np.arange(100).reshape(10,10)
print(A)
lst_of_rows = [1, 2, 4]
lst_of_columns = [1, 2, 5]
sub_matrix = A[lst_of_rows][:, lst_of_columns]
print(sub_matrix)

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]]
[[11 12 15]
 [21 22 25]
 [41 42 45]]


* Be careful, advanced indexing always returns a copy of the data (contrast with basic slicing that returns a view).

In [64]:
B = A[[0, 1, 2], [0, 1, 2]]
print(B)

[ 0 11 22]


In [65]:
print(A)

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]]


In [66]:
B[...] = -1

In [67]:
print(B)

[-1 -1 -1]


In [68]:
print(A)

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]]


 ### Boolean array indexing

* Finding the elements bigger than ...

In [69]:
A = np.arange(20)
print(A, A.shape)
bool_idx = (A>12)
print(bool_idx, bool_idx.shape)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] (20,)
[False False False False False False False False False False False False
 False  True  True  True  True  True  True  True] (20,)


* Printing the elements bigger than ...

In [70]:
print(A[bool_idx])

[13 14 15 16 17 18 19]


* Getting the elements of an array smaller than ...:

In [71]:
A = (100*(0.5-np.random.rand(25))).astype(np.int16).reshape(5,5)
print(A)
print(A[A<0]) # Notice that len(A[A<0]) <= len(A)

[[-33  19 -28 -13 -26]
 [ -7   2  33  28  -5]
 [-39  -8  43  28  40]
 [-21 -17 -46  -4   3]
 [ 41  46 -30  35  30]]
[-33 -28 -13 -26  -7  -5 -39  -8 -21 -17 -46  -4 -30]


* Changing the elements smaller than ...:

In [72]:
A[A<0] = 0
print(A)

[[ 0 19  0  0  0]
 [ 0  2 33 28  0]
 [ 0  0 43 28 40]
 [ 0  0  0  0  3]
 [41 46  0 35 30]]


### Iterating

In [73]:
for row in A:
    print(row)

[ 0 19  0  0  0]
[ 0  2 33 28  0]
[ 0  0 43 28 40]
[0 0 0 0 3]
[41 46  0 35 30]


In [74]:
for element in A.flat:
    print(element, end=' ')

0 19 0 0 0 0 2 33 28 0 0 0 43 28 40 0 0 0 0 3 41 46 0 35 30 

## Extending arrays

* Appending elements:

In [75]:
A = np.arange(3)
print(A)
A = np.append(A, 4)
print(A)

[0 1 2]
[0 1 2 4]


In [76]:
A = np.append(-1, A)
print(A)

[-1  0  1  2  4]


In [77]:
B = np.concatenate((A, A), axis=None)
print(B)

[-1  0  1  2  4 -1  0  1  2  4]


In [78]:
B = np.concatenate((A, A), axis=0)
print(B)

[-1  0  1  2  4 -1  0  1  2  4]


In [79]:
A = np.arange(30).reshape(5,6)
print(A)

[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]]


In [80]:
B = np.ones(5, dtype=np.int16).reshape(5,1)
print(B)

[[1]
 [1]
 [1]
 [1]
 [1]]


In [81]:
C = np.hstack((A, B))
print(C)

[[ 0  1  2  3  4  5  1]
 [ 6  7  8  9 10 11  1]
 [12 13 14 15 16 17  1]
 [18 19 20 21 22 23  1]
 [24 25 26 27 28 29  1]]


In [82]:
B = np.ones(7, dtype=np.int16)
print(B)

[1 1 1 1 1 1 1]


In [83]:
C = np.vstack((C, B))
print(C)

[[ 0  1  2  3  4  5  1]
 [ 6  7  8  9 10 11  1]
 [12 13 14 15 16 17  1]
 [18 19 20 21 22 23  1]
 [24 25 26 27 28 29  1]
 [ 1  1  1  1  1  1  1]]


## Permuting (swapping) dimensions (axes)

* Permuting dimensions only makes sense when the number of dimensions is > 1:

In [84]:
A = np.arange(20).reshape(5,4)
print(A, A.shape)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]] (5, 4)


In [85]:
print(np.transpose(A), np.transpose(A).shape)

[[ 0  4  8 12 16]
 [ 1  5  9 13 17]
 [ 2  6 10 14 18]
 [ 3  7 11 15 19]] (4, 5)


In [86]:
print(A.T, A.T.shape)

[[ 0  4  8 12 16]
 [ 1  5  9 13 17]
 [ 2  6 10 14 18]
 [ 3  7 11 15 19]] (4, 5)


* Transposing permutes all the dimensions:

In [87]:
A = np.arange(60).reshape(3,5,4)
print(A, A.shape)

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]
  [12 13 14 15]
  [16 17 18 19]]

 [[20 21 22 23]
  [24 25 26 27]
  [28 29 30 31]
  [32 33 34 35]
  [36 37 38 39]]

 [[40 41 42 43]
  [44 45 46 47]
  [48 49 50 51]
  [52 53 54 55]
  [56 57 58 59]]] (3, 5, 4)


In [88]:
print(A.T, A.T.shape)

[[[ 0 20 40]
  [ 4 24 44]
  [ 8 28 48]
  [12 32 52]
  [16 36 56]]

 [[ 1 21 41]
  [ 5 25 45]
  [ 9 29 49]
  [13 33 53]
  [17 37 57]]

 [[ 2 22 42]
  [ 6 26 46]
  [10 30 50]
  [14 34 54]
  [18 38 58]]

 [[ 3 23 43]
  [ 7 27 47]
  [11 31 51]
  [15 35 55]
  [19 39 59]]] (4, 5, 3)


## Increasing and decreasing dimensions

* Shape and dimensions:

In [89]:
A = np.arange(5)
print(A, A.shape, A.ndim)

[0 1 2 3 4] (5,) 1


* Increasing the dimensions on the right:

In [90]:
B = A[:, None]
print(B, B.shape, B.ndim)

[[0]
 [1]
 [2]
 [3]
 [4]] (5, 1) 2


* Increasing the dimensions on the left:

In [91]:
B = A[None, :]
print(B, B.shape, B.ndim)

[[0 1 2 3 4]] (1, 5) 2


* For convenience, NumPy provides the np.newaxis object instead None (althougt both are quivalent):

In [92]:
B = A[np.newaxis, :]
print(B, B.shape, B.ndim)

[[0 1 2 3 4]] (1, 5) 2


## Slicing

In [93]:
A = np.arange(10)
print(A)

[0 1 2 3 4 5 6 7 8 9]


In [94]:
print(A[1:3]) # [start:end] (end not included)

[1 2]


In [95]:
print(A[1:5:2]) # [start:end:step]

[1 3]


In [96]:
print(A[:3]) # By default, the first one

[0 1 2]


In [97]:
print(A[3:])

[3 4 5 6 7 8 9]


In [98]:
print(A[::3])

[0 3 6 9]


In [99]:
print(A, A[:], A[::])

[0 1 2 3 4 5 6 7 8 9] [0 1 2 3 4 5 6 7 8 9] [0 1 2 3 4 5 6 7 8 9]


In [100]:
print(A[::-1])

[9 8 7 6 5 4 3 2 1 0]


In [101]:
A = np.arange(50).reshape(5,10)
print(A)
print(A[:]) # <- works, although it is preferable ...
print(A[:,:]) # <- ... this

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]]
[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]]
[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]]


In [102]:
print(A[0:2,:])

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]]


In [103]:
print(A[:,1::2])

[[ 1  3  5  7  9]
 [11 13 15 17 19]
 [21 23 25 27 29]
 [31 33 35 37 39]
 [41 43 45 47 49]]


In [104]:
print(A[A.shape[0]-2:,A.shape[1]-2:]) # bottom-right 2x2 array:

[[38 39]
 [48 49]]


In [105]:
print(A[2]) # Row extraction
print(A[2:3,:]) # Sub-matrix extraction

[20 21 22 23 24 25 26 27 28 29]
[[20 21 22 23 24 25 26 27 28 29]]


* Slices are [views](https://docs.scipy.org/doc/numpy/glossary.html#term-view) of the same data:

In [106]:
A = np.arange(10)
print(A)
B = A[1:3] # B is simply a new view of A
B[:] = 1000
print(B)
print(A)

[0 1 2 3 4 5 6 7 8 9]
[1000 1000]
[   0 1000 1000    3    4    5    6    7    8    9]


In [107]:
A = np.arange(10)
print(A)
B = A[::-1]
B[1] = 1000
print(B)
print(A)

[0 1 2 3 4 5 6 7 8 9]
[   9 1000    7    6    5    4    3    2    1    0]
[   0    1    2    3    4    5    6    7 1000    9]


* Copying slices:

In [108]:
A = np.arange(10)
print(A)
B = A[::-1].copy()
B[1] = 1000
print(B)
print(A)

[0 1 2 3 4 5 6 7 8 9]
[   9 1000    7    6    5    4    3    2    1    0]
[0 1 2 3 4 5 6 7 8 9]


* Ellipsis:

In [109]:
A = np.arange(27).reshape(3,3,3)
print(A)

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]
  [24 25 26]]]


In [110]:
print(A[1,:,:])

[[ 9 10 11]
 [12 13 14]
 [15 16 17]]


In [111]:
print(A[1,...])

[[ 9 10 11]
 [12 13 14]
 [15 16 17]]


## Mathematics
Using vectorization.

### Some *school* math

In [112]:
A = np.random.randint(low=-1, high=+2, size=10)
print(A)
print(-A)

[ 0  0  1  1  0  0  0  0 -1  1]
[ 0  0 -1 -1  0  0  0  0  1 -1]


In [113]:
B = np.random.randint(low=-1, high=+2, size=10)
print(A)
print(B)
print("-"*31)
print(A+B)

[ 0  0  1  1  0  0  0  0 -1  1]
[ 0  0  1  0  1 -1 -1  1  1  0]
-------------------------------
[ 0  0  2  1  1 -1 -1  1  0  1]


In [114]:
print(A*B)
print(A/B)
print(A//2)
print(A>>1)
print(A*2)
print(A**2)
print(1/A)
print(np.absolute(A))

[ 0  0  1  0  0  0  0  0 -1  0]
[nan nan  1. inf  0. -0. -0.  0. -1. inf]
[ 0  0  0  0  0  0  0  0 -1  0]
[ 0  0  0  0  0  0  0  0 -1  0]
[ 0  0  2  2  0  0  0  0 -2  2]
[0 0 1 1 0 0 0 0 1 1]
[inf inf  1.  1. inf inf inf inf -1.  1.]
[0 0 1 1 0 0 0 0 1 1]


/tmp/ipykernel_147728/2526621130.py:2: RuntimeWarning: divide by zero encountered in divide
  print(A/B)
/tmp/ipykernel_147728/2526621130.py:2: RuntimeWarning: invalid value encountered in divide
  print(A/B)
/tmp/ipykernel_147728/2526621130.py:7: RuntimeWarning: divide by zero encountered in divide
  print(1/A)


### Some measurements

In [115]:
A = np.arange(10)
print(A)

[0 1 2 3 4 5 6 7 8 9]


In [116]:
print("Sum =", np.sum(A))
print("Max =", np.max(A))
print("Min =", np.min(A))
print("L0 norm =", np.linalg.norm(A, ord=0)) # np.max(A)
print("L1 norm =", np.linalg.norm(A, ord=1)) # np.sum(A)
print("L2 norm =", np.linalg.norm(A)) # math.sqrt(sum(A_i**2 for A_i in A))
print("L4 norm =", np.linalg.norm(A, ord=4))

Sum = 45
Max = 9
Min = 0
L0 norm = 9.0
L1 norm = 45.0
L2 norm = 16.881943016134134
L4 norm = 11.127735237293699


### Some vector math

In [117]:
from IPython.display import display, Math
A = np.arange(10)
A += 1
print(A)
B = A[::-1]
display(Math(r"A \cdot B={}".format(np.dot(A, B))))
print("A dot product B =", np.dot(A, B))
print("sum(A_i*B_i for A_i, B_i in zip(A, B))=", sum(A_i*B_i for A_i, B_i in zip(A, B)))
print("multiply.reduce(A)=", np.multiply.reduce(A))
print("sum(A[:]*B[:]) =", sum(A[:]*B[:]))

[ 1  2  3  4  5  6  7  8  9 10]


<IPython.core.display.Math object>

A dot product B = 220
sum(A_i*B_i for A_i, B_i in zip(A, B))= 220
multiply.reduce(A)= 3628800
sum(A[:]*B[:]) = 220


In [118]:
A = np.array([1, 2, 3])
B = np.array([-1, -2, -3])
display(Math(r"A \times B={}".format(np.cross(A, B))))
# https://stackoverflow.com/questions/1984799/cross-product-of-two-vectors-in-python
print(f"[A[1]*B[2] - A[2]*B[1], A[2]*B[0] - A[0]*B[2], A[0]*B[1] - A[1]*B[0]] = [{A[1]*B[2] - A[2]*B[1]} {A[2]*B[0] - A[0]*B[2]} {A[0]*B[1] - A[1]*B[0]}]")

<IPython.core.display.Math object>

[A[1]*B[2] - A[2]*B[1], A[2]*B[0] - A[0]*B[2], A[0]*B[1] - A[1]*B[0]] = [0 0 0]


In [119]:
A = np.arange(4).reshape(2,2)
print(A)
B = A[::-1]
print(B)
print("A dot product B = ", np.dot(A, B)) # dot[i,j] = sum(A[i,:] * B[:,j])
# https://stackoverflow.com/questions/11033573/difference-between-numpy-dot-and-inner
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        print(np.sum(A[i,:]*B[:,j]), end=" ")
    print("")
print("A inner product B =", np.inner(A, B)) # inner[i,j] = (A[i,:] * B[j,:])
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        print(np.sum(A[i,:]*B[j,:]), end=" ")
    print("")

[[0 1]
 [2 3]]
[[2 3]
 [0 1]]
A dot product B =  [[0 1]
 [4 9]]
0 1 
4 9 
A inner product B = [[ 3  1]
 [13  3]]
3 1 
13 3 


### Some matrix math

In [120]:
A = np.array([[(i+j)%2 for j in range(10)] for i in range(10)])
print(A, A.shape)

[[0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]] (10, 10)


In [121]:
B = np.array([[1] for i in range(10)])
print(B, B.shape)

[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]] (10, 1)


In [122]:
C = A @ B # Product matrix-matrix
print(C)

[[5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]]


In [123]:
print(C.T, C.T.shape, C.shape) # Transpose

[[5 5 5 5 5 5 5 5 5 5]] (1, 10) (10, 1)


In [124]:
print("Determinant =", np.linalg.det(A))

Determinant = 0.0


In [125]:
R = np.random.rand(5,5)
iR = np.linalg.inv(R)
print("Inverse =", np.linalg.inv(iR)) # Matrix inverse

Inverse = [[0.82875447 0.40136706 0.70743342 0.61775079 0.98096748]
 [0.28507988 0.7074332  0.1334363  0.78349316 0.4288855 ]
 [0.54204968 0.10540028 0.70218125 0.85419529 0.38447409]
 [0.93987128 0.40846562 0.68557188 0.36914891 0.52472433]
 [0.94468593 0.44874689 0.65004402 0.16979571 0.16685066]]


In [126]:
print(np.round(R @ iR))

[[ 1. -0.  0.  0. -0.]
 [ 0.  1. -0. -0.  0.]
 [ 0. -0.  1. -0. -0.]
 [ 0. -0.  0.  1.  0.]
 [ 0. -0.  0. -0.  1.]]


In [127]:
print(R @ iR)

[[ 1.00000000e+00 -1.31440132e-16  1.17832109e-16  1.18478213e-15
  -7.27817418e-16]
 [ 3.81800887e-16  1.00000000e+00 -8.79781998e-18 -2.78279455e-16
   5.35129491e-18]
 [ 7.79103643e-16 -1.90219160e-17  1.00000000e+00 -1.32453634e-15
  -6.10759141e-16]
 [ 1.78231392e-15 -1.76607482e-16  1.65387067e-16  1.00000000e+00
   1.94763787e-15]
 [ 5.84000157e-16 -1.07606075e-16  5.53185583e-17 -1.36345423e-16
   1.00000000e+00]]


In [128]:
print(np.round(iR @ R))

[[ 1.  0.  0.  0. -0.]
 [ 0.  1. -0.  0.  0.]
 [-0. -0.  1.  0. -0.]
 [-0. -0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]


In [129]:
print(iR @ R)

[[ 1.00000000e+00  4.49655871e-16  2.10097637e-15  9.22418521e-17
  -3.64989339e-16]
 [ 1.82027569e-15  1.00000000e+00 -5.16295087e-18  2.49042018e-16
   6.31363061e-16]
 [-1.99941182e-15 -1.57833803e-15  1.00000000e+00  2.72399903e-16
  -1.39282501e-15]
 [-1.57034833e-16 -1.60524641e-16  4.72212208e-16  1.00000000e+00
   1.78241439e-16]
 [ 7.03499165e-17  8.66099166e-17  2.46501110e-16  2.30047082e-17
   1.00000000e+00]]


In [130]:
R = np.random.rand(5,4)
iR = np.linalg.pinv(R) # Pseudo-inverse
print(iR)

[[ 0.30362895 -0.06836897  0.15672257 -0.82913466  1.36759363]
 [ 0.97077993  1.07911751 -0.8687301  -0.12819779 -1.03984351]
 [ 0.45888031 -0.55296682  0.75347955  0.77144651 -1.03477885]
 [-0.97366366  0.55009915 -0.2477474   0.44471609  0.38188209]]


In [131]:
print(np.round(R @ iR))

[[ 1.  0.  0. -0. -0.]
 [ 0.  1. -0.  0.  0.]
 [ 0. -0.  1.  0.  0.]
 [-0.  0.  0.  1. -0.]
 [-0.  0.  0. -0.  1.]]


In [132]:
print(R @ iR)

[[ 0.90793771  0.11857411  0.18223593 -0.16534164 -0.09475871]
 [ 0.11857411  0.84727928 -0.23471568  0.21295623  0.12204703]
 [ 0.18223593 -0.23471568  0.6392667   0.32729132  0.18757345]
 [-0.16534164  0.21295623  0.32729132  0.70305041 -0.17018434]
 [-0.09475871  0.12204703  0.18757345 -0.17018434  0.9024659 ]]


In [133]:
print(np.round(iR @ R))

[[ 1.  0.  0.  0.]
 [-0.  1. -0. -0.]
 [-0. -0.  1. -0.]
 [ 0. -0. -0.  1.]]


In [134]:
print(iR @ R)

[[ 1.00000000e+00  3.37261186e-16  3.31556747e-16  5.40013468e-17]
 [-8.00369111e-17  1.00000000e+00 -1.17494725e-16 -4.80612421e-16]
 [-4.80385703e-16 -1.75827511e-16  1.00000000e+00 -1.16610552e-15]
 [ 8.17752205e-17 -2.28724261e-16 -3.67234047e-16  1.00000000e+00]]


## [Broadcasting](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html)
In vectorized operations, NumPy "extends" scalars and arrays with one of its dimensions equal to the size of the other(s) array(s).

In [135]:
A = np.array([1, 2, 3])
b = 1
print(A+b) # Scalars are broadcasted

[2 3 4]


In [136]:
B = [1]
print(A+B) # When possible, arrays are broadcasted in all possible dimensions

[2 3 4]


In [137]:
A = np.ones((5,3), dtype=np.int16)
print(A)
B = np.arange(3)
print(B)
print(A+B)  # Broadcasting in the axis 0

[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]
[0 1 2]
[[1 2 3]
 [1 2 3]
 [1 2 3]
 [1 2 3]
 [1 2 3]]


In [138]:
print(A)
B = np.arange(5).reshape((5, 1))
print(B)
print(A+B)  # Broadcasting in the axis 1

[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]
[[0]
 [1]
 [2]
 [3]
 [4]]
[[1 1 1]
 [2 2 2]
 [3 3 3]
 [4 4 4]
 [5 5 5]]


* Two dimensions are compatible when they are equal, or
one of them is 1. Otherwise a `ValueError: frames are not aligned` is thrown.

In [139]:
B = np.arange(4)[:, None]
print(B)

[[0]
 [1]
 [2]
 [3]]


In [140]:
print(A.shape)

(5, 3)


In [141]:
print(B.shape)

(4, 1)


In [142]:
try:
    A + B
except ValueError as e:
    print("ValueError exception: ", end='')
    if hasattr(e, 'message'):
        print(e.message)
    else:
        print(e)

ValueError exception: operands could not be broadcast together with shapes (5,3) (4,1) 


## How fast is Numpy's array math?

In [143]:
A = np.array([[(i*10+j) for j in range(10)] for i in range(10)])
print(A, A.shape)

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]] (10, 10)


* Add `B[]` to all the rows of `A[][]` using scalar arithmetic:

In [144]:
C = np.empty_like(A)
B = np.array([(i*10+j) for j in range(10)])
def add():
    for i in range(A.shape[1]):
        for j in range(A.shape[0]):
            C[i, j] = A[i, j] + B[j]
%timeit add()
print(C)

17.2 µs ± 91.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
[[ 10  12  14  16  18  20  22  24  26  28]
 [ 20  22  24  26  28  30  32  34  36  38]
 [ 30  32  34  36  38  40  42  44  46  48]
 [ 40  42  44  46  48  50  52  54  56  58]
 [ 50  52  54  56  58  60  62  64  66  68]
 [ 60  62  64  66  68  70  72  74  76  78]
 [ 70  72  74  76  78  80  82  84  86  88]
 [ 80  82  84  86  88  90  92  94  96  98]
 [ 90  92  94  96  98 100 102 104 106 108]
 [100 102 104 106 108 110 112 114 116 118]]


* Add `B[]` to all the rows of `B[][]` using vectorial computation:

In [145]:
C = np.empty_like(A)
def add():
    for i in range(A.shape[1]):
        C[i, :] = A[i, :] + B
%timeit add()
print(C)

5.91 µs ± 34.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
[[ 10  12  14  16  18  20  22  24  26  28]
 [ 20  22  24  26  28  30  32  34  36  38]
 [ 30  32  34  36  38  40  42  44  46  48]
 [ 40  42  44  46  48  50  52  54  56  58]
 [ 50  52  54  56  58  60  62  64  66  68]
 [ 60  62  64  66  68  70  72  74  76  78]
 [ 70  72  74  76  78  80  82  84  86  88]
 [ 80  82  84  86  88  90  92  94  96  98]
 [ 90  92  94  96  98 100 102 104 106 108]
 [100 102 104 106 108 110 112 114 116 118]]


* Add `B[]` to all the rows of `A[][]` using fully vectorial computation:

In [146]:
%timeit C = A + B # <- broadcasting is faster
print(C)

667 ns ± 3.23 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
[[ 10  12  14  16  18  20  22  24  26  28]
 [ 20  22  24  26  28  30  32  34  36  38]
 [ 30  32  34  36  38  40  42  44  46  48]
 [ 40  42  44  46  48  50  52  54  56  58]
 [ 50  52  54  56  58  60  62  64  66  68]
 [ 60  62  64  66  68  70  72  74  76  78]
 [ 70  72  74  76  78  80  82  84  86  88]
 [ 80  82  84  86  88  90  92  94  96  98]
 [ 90  92  94  96  98 100 102 104 106 108]
 [100 102 104 106 108 110 112 114 116 118]]


## Vectorized conditionals using [masked arrays](https://numpy.org/doc/stable/reference/maskedarray.generic.html)

In [147]:
import numpy as np
import time

a = np.arange(1_000_000)
t1 = time.time()
for i in range(len(a)):
    if a[i] > 1000 and a[i] < 2000:
        a[i] = a[i] / 1000
t2 = time.time()
print("Using condiditional:", t2 - t1)

a = np.arange(1_000_000)
t1 = time.time()
mask = np.ma.masked_inside(a, 1000, 2000) # Mask an array inside a given interval
a[mask] = a[mask] / 1000
t2 = time.time()
print("Using mask:", t2 - t1)

Using condiditional: 0.14004898071289062
Using mask: 0.010968685150146484


## Arrays of objects

* For example, an array of strings:

In [148]:
A = np.array(['hello', 'world!'])
print(A)
print(A.shape)
print(np.char.upper(A))

['hello' 'world!']
(2,)
['HELLO' 'WORLD!']


* Simulating a dictionary:

In [149]:
A = np.array([("Spain", 100), ("France", 200), ("Italy", 300)])
print(A) # Notice that all the elements are srings
print(A.shape)
print(A[:,0])
print(A[A[:,0] == "France"])
print(A[A[:,0] == "France"][:,1])
print("The value associated to the key France is", A[A[:,0] == "France"][:,1][0])

[['Spain' '100']
 ['France' '200']
 ['Italy' '300']]
(3, 2)
['Spain' 'France' 'Italy']
[['France' '200']]
['200']
The value associated to the key France is 200


* A dictionary is faster:

In [150]:
%timeit A[A[:,0] == "France"][:,1][0]
dictionary = {"Spain":100, "France":200, "Italy":300}
print(dictionary["France"])
%timeit dictionary["France"]

1.68 µs ± 5.12 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
200
15.1 ns ± 0.089 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


* However, this difference can be smaller depending on the type of search:

In [151]:
others = [value for key, value in dictionary.items() if key != "France"]
print(others)
%timeit [value for key, value in dictionary.items() if key != "France"]

[100, 300]
178 ns ± 2.42 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [152]:
print(A[:,0] != "France")
print(A[A[:,0] != "France"])
print(A[A[:,0] != "France"][:,1])
print(A[A[:,0] != "France"][:,1].astype(np.int16))
%timeit A[A[:,0] != "France"][:,1].astype(np.int16)

[ True False  True]
[['Spain' '100']
 ['Italy' '300']]
['100' '300']
[100 300]
2.44 µs ± 19.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Structured arrays

* Create a 1D array of (two) records, where each record has the structure (int, float, char[10]).

In [153]:
X = np.array([(1, 2., "Hello"), (3, 4., "World")],
             dtype=[("first", "i4"),("second", "f4"), ("third", "S10")])
# See https://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html
print(X)

[(1, 2., b'Hello') (3, 4., b'World')]


* Get the first element of every record:

In [154]:
print(X["first"])

[1 3]


* Get the first record:

In [155]:
print(X[0])

(1, 2., b'Hello')


* Get the second element of every record:

In [156]:
print(X["second"])

[2. 4.]


* Third element of every record:

In [157]:
print(X["third"])

[b'Hello' b'World']


## Disk I/O

* Output data to an ASCII file:

In [158]:
Data = np.array([[1., 200.], [2., 150.], [3., 250.]])
np.savetxt("data.txt", Data)
!cat Data.txt

cat: Data.txt: No such file or directory


* Input data from an ASCII file:

In [159]:
np.genfromtxt('data.txt')

array([[  1., 200.],
       [  2., 150.],
       [  3., 250.]])

* Output data to a binary file (using the native endianness):

In [160]:
ofile = open("data.float64", mode="wb")
Data.tofile(ofile)

* Input data from a binary file (using the native endianness):

In [161]:
print(np.fromfile("data.float64", dtype=np.float64))

[  1. 200.   2. 150.   3. 250.]


* Numpy and C use the same endianness:

In [162]:
!cat create_float64.c
!gcc create_float64.c -o create_float64
!./create_float64

#include <stdio.h>

#define N 10

int main() {
  double a[N];
  int i;
  FILE *ofile = fopen("data.float64", "wb");
  for(i=0; i<N; i++) {
    a[i] = i;
  }
  fwrite(a, sizeof(double), N, ofile);
  fclose(ofile);
  fprintf(stderr,"create_float64: done\n");
}
create_float64: done


In [163]:
print(np.fromfile("data.float64", dtype=np.float64))

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


* Specifiying the endianness:

In [164]:
print(np.fromfile("data.float64", dtype=">d"))
# (> = bit-endian, d = double, see https://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html)

[0.00000e+000 3.03865e-319 3.16202e-322 1.04347e-320 2.05531e-320
 2.56124e-320 3.06716e-320 3.57308e-320 4.07901e-320 4.33197e-320]


* Make the things easier:

In [165]:
ofile = open("data.npy", mode="wb")
A = (100*np.random.rand(2,3)).astype(np.uint16)
print(A)

[[56 85 34]
 [78 95 41]]


In [166]:
np.save(ofile, A)

In [167]:
!ls save*

ls: cannot access 'save*': No such file or directory


In [168]:
print(np.load("data.npy"))

[[56 85 34]
 [78 95 41]]


### [Endianness](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dtype.byteorder.html)

In [169]:
print(A.dtype.byteorder)

=


## Timming NumPy?

* Lets define a list and compute the sum of its elements, timing it:

In [170]:
l = list(range(0,100000)); print(type(l), l[:10])
%timeit sum(l)

<class 'list'> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
833 µs ± 56.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


* An now, lets create a numpy's array and time the sum of its elements:

In [171]:
A = np.arange(0, 100000); print(type(A), A[:10])
%timeit np.sum(A)

<class 'numpy.ndarray'> [0 1 2 3 4 5 6 7 8 9]
23.2 µs ± 33.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


* And what about a *pure* C implementation of an equivalent computation: 

In [172]:
!cat sum_array.c
!gcc -O3 sum_array.c -o sum_array
%timeit !./sum_array

#include <stdio.h>
#include <time.h>
#include "sum_array_lib.c"

#define N 100000

int main() {
  double a[N];
  int i;
  clock_t start, end;
  double cpu_time;
  for(i=0; i<N; i++) {
    a[i] = i;
  }
  start = clock();
  double sum = sum_array(a,N);
  end = clock();
  printf("%f ", sum);
  cpu_time = ((double) (end - start)) / CLOCKS_PER_SEC;
  cpu_time *= 1000000;
  printf("%f usegs\n", cpu_time);
}
4999950000.000000 135.000000 usegs
4999950000.000000 156.000000 usegs
4999950000.000000 156.000000 usegs
4999950000.000000 134.000000 usegs
4999950000.000000 46.000000 usegs
4999950000.000000 135.000000 usegs
4999950000.000000 46.000000 usegs
4999950000.000000 155.000000 usegs
4999950000.000000 126.000000 usegs
4999950000.000000 112.000000 usegs
4999950000.000000 125.000000 usegs
4999950000.000000 50.000000 usegs
4999950000.000000 119.000000 usegs
4999950000.000000 155.000000 usegs
4999950000.000000 136.000000 usegs
4999950000.000000 131.000000 usegs
4999950000.000000 151.000000 usegs
49

* Another example:

In [173]:
# Example extracted from https://github.com/pyHPC/pyhpc-tutorial
lst = range(1000000)

for i in lst[:10]:
    print(i, end=' ')
print()

%timeit [i + 1 for i in lst] # A Python list comprehension (iteration happens in C but with PyObjects)
x = [i + 1 for i in lst]

print(x[:10])

arr = np.arange(1000000) # A NumPy list of integers
%timeit arr + 1 # Use operator overloading for nice syntax, now iteration is in C with ints
y = arr + 1

print(y[:10])

0 1 2 3 4 5 6 7 8 9 
31.5 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
300 µs ± 2.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
[ 1  2  3  4  5  6  7  8  9 10]
